In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

/Users/muratyalcin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(train.shape)
print(test.shape)
print(sample.shape)

(903653, 12)
(804684, 12)
(617242, 2)


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
channelGrouping         903653 non-null object
date                    903653 non-null int64
device                  903653 non-null object
fullVisitorId           903653 non-null object
geoNetwork              903653 non-null object
sessionId               903653 non-null object
socialEngagementType    903653 non-null object
totals                  903653 non-null object
trafficSource           903653 non-null object
visitId                 903653 non-null int64
visitNumber             903653 non-null int64
visitStartTime          903653 non-null int64
dtypes: int64(4), object(8)
memory usage: 82.7+ MB


In [5]:
train.describe()

,date,visitId,visitNumber,visitStartTime
count,9.036530e+05,9.036530e+05,903653.000000,9.036530e+05
mean,2.016589e+07,1.485007e+09,2.264897,1.485007e+09
std,4.697698e+03,9.022124e+06,9.283735,9.022124e+06
min,2.016080e+07,1.470035e+09,1.000000,1.470035e+09
25%,2.016103e+07,1.477561e+09,1.000000,1.477561e+09
50%,2.017011e+07,1.483949e+09,1.000000,1.483949e+09
75%,2.017042e+07,1.492759e+09,1.000000,1.492759e+09
max,2.017080e+07,1.501657e+09,395.000000,1.501657e+09


In [6]:
train.channelGrouping.unique()

array(['Organic Search', 'Referral', 'Paid Search', 'Affiliates',
       'Direct', 'Display', 'Social', '(Other)'], dtype=object)

In [8]:
len(train.fullVisitorId.unique())

742735

In [10]:
type(train.date[0]) # convert datetime

numpy.int64

In [11]:
train.device[0] # browser(dene), isMobile

'{"browser": "Chrome", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}'

In [13]:
train.geoNetwork[100] # continent, country, subcontinent

'{"continent": "Americas", "subContinent": "South America", "country": "Argentina", "region": "Buenos Aires", "metro": "(not set)", "city": "Buenos Aires", "cityId": "not available in demo dataset", "networkDomain": "phonevision.com.ar", "latitude": "not available in demo dataset", "longitude": "not available in demo dataset", "networkLocation": "not available in demo dataset"}'

In [ ]:
train.sessionId  # get rid of

In [15]:
train.socialEngagementType.unique() # get rid of

array(['Not Socially Engaged'], dtype=object)

In [16]:
train[train.socialEngagementType=="Socially Engaged"]

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime


In [23]:
train.totals[100]

'{"visits": "1", "hits": "1", "pageviews": "1", "bounces": "1", "newVisits": "1"}'

In [22]:
len(train.totals.unique())

14840

In [26]:
train.trafficSource[1000]

'{"campaign": "(not set)", "source": "google", "medium": "organic", "keyword": "(not provided)", "adwordsClickInfo": {"criteriaParameters": "not available in demo dataset"}, "isTrueDirect": true}'

In [27]:
train.visitNumber.unique()

array([  1,   2,   3,   5,  11,   4,  57,   6,  56,   7,  20,   8,  15,
         9,  25,  24,  14,  89, 136,  13,  85,  10, 105,  22,  21,  48,
        18,  26,  17,  83,  38,  84,  27,  42, 100,  31,  16,  30,  59,
        50,  19,  51,  29, 160,  52,  12,  63,  23,  47,  49,  28,  34,
       178,  88,  33, 317,  93, 140,  35,  41,  91,  92,  74, 309,  43,
        61,  46,  45,  32, 108,  86, 195,  36, 236,  72, 162, 235,  90,
        76,  77,  94,  96,  73,  78,  97,  87, 304, 106, 107, 389,  68,
        98, 132,  58, 266, 267, 221,  39, 121, 143, 142, 174, 175, 189,
       191,  55, 190, 104, 245, 204,  37,  82, 137, 206, 101,  62, 110,
       156, 158, 157, 159, 138, 109, 194, 193, 147, 254,  71,  53, 144,
       145, 146,  80, 253,  99, 134, 135, 102,  44, 130,  95,  75, 103,
       315,  70,  69, 117, 163, 154, 153, 283,  65, 262, 263, 295, 297,
       298, 219,  64, 296, 155, 141, 169, 170,  60, 207,  54, 187, 133,
       118, 264,  67,  66, 126,  40, 122, 325, 326, 197, 167, 19

In [29]:
type(train.visitStartTime[0])

numpy.int64

In [4]:
train.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [ ]:
# Data Cleaning

In [ ]:
# Exploratory Analysis

In [ ]:
# Feature Selection

In [ ]:
# Model Fitting

In [ ]:
# Grid Search

In [ ]:
# Prediction

In [5]:
test.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6167871330617112363,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",6167871330617112363_1508151024,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""4""}","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508151024,2,1508151024
1,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0643697640977915618,"{""continent"": ""Europe"", ""subContinent"": ""South...",0643697640977915618_1508175522,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508175522,1,1508175522
2,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6059383810968229466,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",6059383810968229466_1508143220,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""7"", ""pageviews"": ""7"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508143220,1,1508143220
3,Organic Search,20171016,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",2376720078563423631,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",2376720078563423631_1508193530,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""4"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508193530,1,1508193530
4,Organic Search,20171016,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",2314544520795440038,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",2314544520795440038_1508217442,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""9"", ""pageviews"": ""4"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508217442,1,1508217442


In [6]:
sample.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.0
1,0000049363351866189,0.0
2,0000053049821714864,0.0
3,0000059488412965267,0.0
4,0000085840370633780,0.0


In [7]:
list(train.columns)==list(test.columns)

True

In [14]:
type(train.totals[10])

str

In [17]:
a = np.random.randint(1,20,50)
a

array([ 3, 18, 16,  9,  2, 16,  1, 17, 13,  9, 14,  5,  6,  9,  7, 16,  3,
        3, 18,  8, 10,  3, 13, 18, 12, 19,  4, 13,  8,  2, 10,  4, 17,  6,
        2, 12, 15, 10,  4,  4,  1, 11,  5, 15,  4,  7, 12,  1, 11, 19])

In [20]:
from collections import deque
b = deque(a)
b.rotate(1)     

In [22]:
c=a+b
c

array([22, 21, 34, 25, 11, 18, 17, 18, 30, 22, 23, 19, 11, 15, 16, 23, 19,
        6, 21, 26, 18, 13, 16, 31, 30, 31, 23, 17, 21, 10, 12, 14, 21, 23,
        8, 14, 27, 25, 14,  8,  5, 12, 16, 20, 19, 11, 19, 13, 12, 30])

In [24]:
c[0]=0
c

array([ 0, 21, 34, 25, 11, 18, 17, 18, 30, 22, 23, 19, 11, 15, 16, 23, 19,
        6, 21, 26, 18, 13, 16, 31, 30, 31, 23, 17, 21, 10, 12, 14, 21, 23,
        8, 14, 27, 25, 14,  8,  5, 12, 16, 20, 19, 11, 19, 13, 12, 30])